In [0]:
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, LongType, DoubleType

schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", LongType(), True),
    StructField("category_id", LongType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("user_id", LongType(), True),
    StructField("user_session", StringType(), True)
])

In [0]:
df_october = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",header="true",schema=schema)

df_november = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",header="true",schema=schema)

In [0]:
df_october.createOrReplaceTempView("df_october")
df_november.createOrReplaceTempView("df_november")

In [0]:
%sql
show tables

database,tableName,isTemporary
default,events,false
default,events_sql,false
,df_november,true
,df_october,true


In [0]:
%sql

CREATE OR REPLACE TABLE workspace.ecommerce.df_october AS
SELECT * FROM df_october;
CREATE OR REPLACE TABLE workspace.ecommerce.df_november AS
SELECT * FROM df_november;

num_affected_rows,num_inserted_rows


In [0]:
df_purchase = spark.sql("""
    SELECT * FROM df_november WHERE event_type = 'purchase' AND brand IS NOT NULL
    UNION ALL
    SELECT * FROM df_october WHERE event_type = 'purchase' AND brand IS NOT NULL
""")

In [0]:
df_view = spark.sql("""
    SELECT * FROM df_november WHERE event_type = 'view' AND brand IS NOT NULL
    UNION ALL
    SELECT * FROM df_october WHERE event_type = 'view' AND brand IS NOT NULL
""")

In [0]:
df_cart = spark.sql("""
    SELECT * FROM df_november WHERE event_type = 'cart' AND brand IS NOT NULL
    UNION ALL
    SELECT * FROM df_october WHERE event_type = 'cart' AND brand IS NOT NULL
""")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

window_spec1 = Window.orderBy(desc("P_total_price"))
TopPurchase = df_purchase.groupBy("brand").agg(sum("price").alias("P_total_price")).withColumn("P_rank", dense_rank().over(window_spec1))

window_spec2 = Window.orderBy(desc("C_total_price"))
TopCart = df_cart.groupBy("brand").agg(sum("price").alias("C_total_price")).withColumn("C_rank", dense_rank().over(window_spec2))

window_spec3 = Window.orderBy(desc("V_total_price"))
TopView = df_view.groupBy("brand").agg(sum("price").alias("V_total_price")).withColumn("V_rank", dense_rank().over(window_spec3))

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
Top = (
    TopPurchase
    .join(TopCart, on="brand", how="left")
    .join(TopView, on="brand", how="left")
)
Top.printSchema

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


<bound method DataFrame.printSchema of DataFrame[brand: string, P_total_price: double, P_rank: int, C_total_price: double, C_rank: int, V_total_price: double, V_rank: int]>

In [0]:
Top_final = Top.withColumn("Cart_conversionRate", (col("P_total_price") / col("C_total_price")) * 100) \
   .withColumn("View_conversionRate", (col("P_total_price") / col("V_total_price")) * 100) \
   .orderBy(col("P_rank")).limit(10)
     

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
Top10purchaser = Top_final.select("brand", "P_total_price", "P_rank", "Cart_conversionRate", "View_conversionRate").orderBy(col("P_rank")).limit(10)
display(Top10purchaser)

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


brand,P_total_price,P_rank,Cart_conversionRate,View_conversionRate
apple,2.3872179369999868E8,1,44.35118605078363,3.050648054249483
samsung,1.012774134799987E8,2,41.45659691274691,2.4062382629510775
xiaomi,2.0453899249999996E7,3,33.886711166210155,1.3797939618274575
huawei,9664104.089999994,4,39.848804814277436,1.528373511526913
lg,8626906.719999999,5,33.410429070885,1.1726570382154273
acer,6924026.050000001,6,42.75615569910694,1.0266211564725058
lucente,6651658.940000005,7,70.35246749849203,1.5222355370050136
sony,6341082.979999996,8,37.35916437726607,1.217175892336623
oppo,5901500.519999993,9,38.85927018726811,1.5906142196678654
lenovo,4450744.829999999,10,40.83500249591307,0.7763427526725397


In [0]:
Top10CartConverser = Top_final.select("brand", "C_total_price", "C_rank", "Cart_conversionRate", "View_conversionRate").orderBy(col("Cart_conversionRate").desc()).limit(10)
display(Top10CartConverser)

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


brand,C_total_price,C_rank,Cart_conversionRate,View_conversionRate
lucente,9454762.819999995,10,70.35246749849203,1.5222355370050136
apple,5.382534605199826E8,1,44.35118605078363,3.050648054249483
acer,1.6194220310000004E7,7,42.75615569910694,1.0266211564725058
samsung,2.4429746052999905E8,2,41.45659691274691,2.4062382629510775
lenovo,1.089933772000001E7,9,40.83500249591307,0.7763427526725397
huawei,2.4251929600000046E7,5,39.848804814277436,1.528373511526913
oppo,1.518685372000004E7,8,38.85927018726811,1.5906142196678654
sony,1.6973299819999974E7,6,37.35916437726607,1.217175892336623
xiaomi,6.035964702999986E7,3,33.886711166210155,1.3797939618274575
lg,2.5820999489999913E7,4,33.410429070885,1.1726570382154273


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
Top10ViewConverser = Top_final.select("brand", "V_total_price", "V_rank", "Cart_conversionRate", "View_conversionRate").orderBy(col("View_conversionRate").desc()).limit(10)
display(Top10ViewConverser)

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


brand,V_total_price,V_rank,Cart_conversionRate,View_conversionRate
apple,7.825281364969803E9,1,44.35118605078363,3.050648054249483
samsung,4.208951999449517E9,2,41.45659691274691,2.4062382629510775
oppo,3.710202289799898E8,13,38.85927018726811,1.5906142196678654
huawei,6.323129795899907E8,6,39.848804814277436,1.528373511526913
lucente,4.3696647320999295E8,9,70.35246749849203,1.5222355370050136
xiaomi,1.4823879373200028E9,3,33.886711166210155,1.3797939618274575
sony,5.2096685613999176E8,8,37.35916437726607,1.217175892336623
lg,7.356717641100416E8,4,33.410429070885,1.1726570382154273
acer,6.744480187600181E8,5,42.75615569910694,1.0266211564725058
lenovo,5.732963713100207E8,7,40.83500249591307,0.7763427526725397


In [0]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, col

def conversion_flag(p_rank, cart_rate, view_rate):
    if p_rank <= 3 and cart_rate >= 40 and view_rate >= 2:
        return "Top Performer"
    elif cart_rate >= 60:
        return "Strong Cart Converter"
    elif view_rate >= 2:
        return "Strong View Converter"
    else:
        return "Needs Improvement"

flag_udf = udf(conversion_flag, StringType())

# Add the Performance_Flag column
Top_final_flagged = Top_final.withColumn(
    "Performance_Flag",
    flag_udf(col("P_rank"), col("Cart_conversionRate"), col("View_conversionRate"))
)

# Now order by P_rank at the DataFrame level
Top_final_ordered = Top_final_flagged.orderBy(col("P_rank"))

display(
    Top_final_ordered.select(
        "brand",
        "P_total_price",
        "P_rank",
        "Cart_conversionRate",
        "View_conversionRate",
        "Performance_Flag"
    )
)

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


brand,P_total_price,P_rank,Cart_conversionRate,View_conversionRate,Performance_Flag
apple,2.3872179369999868E8,1,44.35118605078363,3.050648054249483,Top Performer
samsung,1.012774134799987E8,2,41.45659691274691,2.4062382629510775,Top Performer
xiaomi,2.0453899249999996E7,3,33.886711166210155,1.3797939618274575,Needs Improvement
huawei,9664104.089999994,4,39.848804814277436,1.528373511526913,Needs Improvement
lg,8626906.719999999,5,33.410429070885,1.1726570382154273,Needs Improvement
acer,6924026.050000001,6,42.75615569910694,1.0266211564725058,Needs Improvement
lucente,6651658.940000005,7,70.35246749849203,1.5222355370050136,Strong Cart Converter
sony,6341082.979999996,8,37.35916437726607,1.217175892336623,Needs Improvement
oppo,5901500.519999993,9,38.85927018726811,1.5906142196678654,Needs Improvement
lenovo,4450744.829999999,10,40.83500249591307,0.7763427526725397,Needs Improvement
